In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 200)
pd.options.display.float_format = '{:,.0f}'.format
from __future__ import division

In [22]:
columns = ['id','scrape_id', 'neighbourhood', 'zipcode', 'property_type', 'room_type', 'accommodates', 'bedrooms', 'beds', 'price', 'number_of_reviews', 'review_scores_rating', 'reviews_per_month', 'bed_type'] 
listings = pd.read_csv('listings.csv')
listings1 = listings.copy()
listings = listings[columns]
listings.index = listings['id']
del listings['id']
listings['price'] = listings['price'].replace(['[\$]', '[\,]'], '', regex=True).apply(pd.to_numeric)
# --> filter zipcode on first 4 numbers
listings['zipcode'] = listings['zipcode'].str[:4].apply(pd.to_numeric, errors='coerce')
listings.dropna(subset=['zipcode'], inplace=True)
# convert dat zipcode to a prupper int
listings['zipcode'] = listings['zipcode'].apply(lambda x: int(x))


listings['zipcode'].iloc[0]

1054

In [3]:
#price & reviews 
listings['zipcode'].astype('category')
price_reviews = pd.pivot_table(listings, index=['zipcode'], values=['price', 'scrape_id','number_of_reviews', 'review_scores_rating', 'reviews_per_month'], aggfunc={'price': np.mean, 'scrape_id': len, 'review_scores_rating': np.mean, 'reviews_per_month': np.mean, 'number_of_reviews': np.sum})
del price_reviews['scrape_id']
# --->>> JSON
price_reviews.sort() # SORT
price_reviews.to_json('price_reviews.json', orient='index')

/Users/sjoerd/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(....) is deprecated, use sort_index(.....)


In [155]:
# PROPERTY TYPES PIVOT

property_types = pd.pivot_table(listings, index='zipcode', columns='property_type', values='scrape_id', aggfunc=len).fillna(value=0)
property_types = pd.DataFrame(property_types)

property_types['total'] = property_types.sum(axis=1)
#property_types = property_types.loc[:, 'Apartment':'Villa'].div(property_types.total, axis = 0)
# --->>> JSON
p_columns = ['Apartment', 'Bed & Breakfast', 'Boat', 'Condominium', 'House', 'Loft', 'Townhouse']
property_types[p_columns].to_json('property_types.json', orient='index')

In [156]:
price_reviews

,number_of_reviews,price,review_scores_rating,reviews_per_month
zipcode,,,,
0,1,120,97,0
173,21,150,96,2
555,12,110,98,1
689,3,310,100,0
1000,0,74,nan,nan
1010,34,125,95,1
1011,7238,177,94,2
1012,13247,180,92,2
1013,11779,142,94,1


In [59]:
bed_types = listings[['zipcode', 'bed_type']]
bed_types['bed_type'].value_counts()

Real Bed         14323
Pull-out Sofa      136
Futon               44
Airbed              11
Couch                8
Name: bed_type, dtype: int64

In [61]:
bed_types['real_bed'] = bed_types['bed_type'].apply(lambda x: if x == 'real_bed': return 1)

SyntaxError: invalid syntax (<ipython-input-61-b5ce61095c69>, line 1)

In [46]:
pd.pivot_table(listings[['zipcode', 'bed_type']], index=['zipcode'], values='bed_type', aggfunc=len)

zipcode
0         1
173       1
555       1
689       1
1000      2
1010      1
1011    281
1012    523
1013    571
1014      9
1015    536
1016    500
1017    626
1018    506
1019    257
1021    119
1022     61
1023     83
1024     41
1025     94
1026     16
1027     16
1028     17
1031     48
1032     88
1033     66
1034     62
1035     65
1036     30
1037      1
       ... 
1087    187
1090      1
1091    483
1092    157
1093    196
1094    358
1095    185
1096     84
1097    110
1098    165
1101      1
1102     72
1103     62
1104     42
1105      1
1106     35
1107     10
1108     31
1109      1
1111      1
1112      1
1115      1
1165      1
1171      2
1175      2
1181      2
1182      1
1186      1
1954      1
2374      1
Name: bed_type, dtype: int64

In [70]:
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar', 'foo', 'foo'],
                   'B' : ['one', 'one', 'two', 'three',
                          'two', 'two', 'one', 'three'],
                   'C' : np.random.randn(8),
                   'D' : np.random.randn(8)})

def get_letter_type(letter):
    if letter.lower() in 'aeiou':
        return 'vowel'
    else:
        return 'consonant'
df

,A,B,C,D
0,foo,one,2,0
1,bar,one,-0,0
2,foo,two,1,1
3,bar,three,0,-1
4,foo,two,-0,-1
5,bar,two,1,-2
6,foo,one,1,-0
7,foo,three,1,-0


In [64]:
grouped = df.groupby(get_letter_type, axis=1)

In [71]:
grouped.sum()

,consonant,vowel
0,1,foo
1,-1,bar
2,1,foo
3,1,bar
4,-1,foo
5,1,bar
6,1,foo
7,3,foo
